All the questions are answered in this notebook itself as part of DragonFruit AI challenge. The notebook has supporting text and comments for explanation and all the code listed below are runnable. </br>
Thanks for providing me with this opportunity

## Question 1

#### RLE for Microscope Image

The microscope image of the parasite is defined as a contiguous blob but still an arbitrary shape, the best after-data structure that I could come up with after much research is Run Length Encoding. </br>
Given a string such as MIIICCCRROO - can be represented as M1I3C3R2O2 when we apply RLE, which is basically we keep track of repeated continuous elements followed by length it occupies. In our case as we have continuous representation and that two just two values 0(background) and 1(parasite), It makes the storage of such a large image highly memory efficient. Now let's look at the Best and Worst Time complexity.</br>
Best Case : </br>
Assuming the best case as a more realistically contiguous area rather than a perfect square, with 2 entries per row for 100,000 rows:
8 bytes * 2 entries/row * 100,000 rows = 1,600,000 bytes or approximately 1.53 MB. </br>
Worst Case : </br>
If we estimate, for example, that the parasite winds through each row of the image, it could potentially double the entries needed,  there will be 100,000 elements in the RLE data - </br>
8 bytes * 4 entries/row * 100,000 rows = 3,200,000 bytes or approximately 3.05 MB. This is still an efficient representation given the image's size 100000x100000.


In [25]:
def rle_encode(image):
    
    encoded_image = {}
    for row_idx, row in enumerate(image):
        encoded_row = []
        prev_value = row[0]
        start = 0  # Starting index of a sequence
        for i in range(1, len(row)):
            if row[i] != prev_value:
                if prev_value == 1:  # Encode sequences of '1's
                    encoded_row.append((start, i - start))
                start = i  # Update the start for the new sequence
                prev_value = row[i]
        # Handle the last sequence in the row
        if prev_value == 1:
            encoded_row.append((start, len(row) - start))
        # Only add non-empty encoded rows
        if encoded_row:
            encoded_image[row_idx] = encoded_row
    
    return encoded_image

If the image 100,000 x 100,000 was provided in form of Tiff or available as an matrix hosted on cloud this is how I would have handled. Though RLE is efficient method to optimize storage, but when you apply for it first time and in worst case you gotta traverse each row of image, the code will run into TLE hence better to parrallize the rle_encoding function thats what here i have demonstrated. Similar approach I have taken for Dye image

#### Parallelization RLE encoding if we were given image straighaway on cloud or as Tiff and had to convert to RLE

We will divide entire image into segments of heights 10,000 and iterating rows, for each segment we will perform RLE encoding parallely

In [13]:
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
import time

# Setup basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def rle_encode_segment(image, start_row, segment_height):
  
    rle_segment = {}
    end_row = start_row + segment_height
    
    for row_idx in range(start_row, min(end_row, len(image))):
        row = image[row_idx]
        if len(row) and np.sum(row)==0:  # Skip empty rows
            continue
        
        encoded_row = []
        prev_value = row[0]
        length = 1
        
        for i in range(1, len(row)):
            if row[i] == prev_value:
                length += 1
            else:
                if prev_value == 1:  # Only encode sequences of 1s
                    encoded_row.append((i - length, length))
                prev_value = row[i]
                length = 1
        # Handle the last sequence in the row
        if prev_value == 1:
            encoded_row.append((len(row) - length, length))
        
        if encoded_row:  # Only add non-empty encoded rows
            rle_segment[row_idx] = encoded_row
    
    return rle_segment



def main():
    image_height=100000
    image_width=1000000
    segment_height=10000
    # DUMMY IMAGE
    img = np.zeros((100000, 100000), dtype=np.uint8)
    img[6000, 2000:2500] = 1
    img[14000, 600:800] = 1
    img[99997, 70000:95000] = 1
    segment_starts = range(0, image_height, segment_height)
    combined_rle_data = {}

    with ProcessPoolExecutor(max_workers=5) as executor:
        ## parallelize rle_encode_segment
        futures = {executor.submit(rle_encode_segment, img, start, segment_height): start for start in segment_starts}
        
        for future in as_completed(futures):
            start_row = futures[future]
            try:
                data = future.result()
                combined_rle_data.update(data)
                logging.info(f"Segment starting at row {start_row} processed successfully.")
            except Exception as exc:
                logging.error(f"Segment starting at row {start_row} generated an error: {exc}")

    return combined_rle_data


if __name__ == '__main__':
    combined_rle_data = main()
    print(combined_rle_data)

2024-03-28 21:54:51,721 - INFO - Segment starting at row 0 processed successfully.
2024-03-28 21:55:30,084 - INFO - Segment starting at row 10000 processed successfully.
2024-03-28 21:55:57,674 - INFO - Segment starting at row 20000 processed successfully.
2024-03-28 21:56:29,229 - INFO - Segment starting at row 30000 processed successfully.
2024-03-28 21:57:00,590 - INFO - Segment starting at row 40000 processed successfully.
2024-03-28 21:57:31,816 - INFO - Segment starting at row 50000 processed successfully.
2024-03-28 21:58:01,873 - INFO - Segment starting at row 60000 processed successfully.
2024-03-28 21:58:38,887 - INFO - Segment starting at row 70000 processed successfully.
2024-03-28 21:59:13,691 - INFO - Segment starting at row 80000 processed successfully.
2024-03-28 21:59:34,451 - INFO - Segment starting at row 90000 processed successfully.
2024-03-28 21:59:35,190 - INFO - Execution time: 330.87690258026123 seconds


{6000: [(2000, 500)], 14000: [(600, 200)], 99997: [(70000, 25000)]}


#### COO for Dye Image

Given the dye image is not necessarily continuous and occupy very less space in the entire image, its nearly sparse. So a better data structure to represent it will be a COO (Coordinate List). That is for each cell value as 1, will be saved as (row,col) in a list or a dictionary where keys are rows and cols as values which are lit/dyed. </br>

Row index: 4 bytes (32-bit integer)
Column index: 4 bytes (32-bit integer)
Total per non-zero element: 4 (row index) + 4 (column index) = 8 bytes </br>
Worst Case - For a fully dense 100,000x100,000 image where every element is non-zero, the total number of non-zero elements equals the total number of elements in the matrix, which is 100,000 * 100,000 = 10,000,000,000 non-zero elements.
The storage requirement would be 8 bytes * 10,000,000,000 = 80,000,000,000 bytes, or approximately 74.51 GB.

In [16]:
import numpy as np

def COO(image):
    COO_dict = {}
    for row_index in range(image.shape[0]):
        cols = np.where(image[row_index] == 1)[0].tolist()
        if cols:  # Only add to dictionary if there are columns with value 1
            COO[row_index] = cols
    return COO_dict



#### Parallelization to Convert given image via tiff or hosted on cloud to COO Data Structure form

We will divide entire image into segments of heights 10,000 and iterating rows, for each segment we will perform COO encoding parallely

In [20]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import logging
 

# Setup basic configuration for logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def find_non_zero_in_segment(segment):
    non_zero_indices = np.argwhere(segment)
    return non_zero_indices

def main():
    """Divide the matrix into horizontal segments and find non-zero elements in parallel."""
    
    num_segments = 10
    image_size = 100000
    #DUMMY IMAGE (Image on cloud or tiff file)
    image = np.zeros((image_size, image_size), dtype=np.uint8)

    # Number of ones to place in the image
    num_ones = 1000  # Example: placing 1000 ones

    # Generate random positions for the ones
    rows = np.random.randint(0, image_size, size=num_ones)
    cols = np.random.randint(0, image_size, size=num_ones)

    # Place the ones in the image
    image[rows, cols] = 1
    matrix = image
    segment_height = matrix.shape[0] // num_segments
    segments = [(i * segment_height, (i + 1) * segment_height) for i in range(num_segments)]

    results = []
    with ProcessPoolExecutor(max_workers=1) as executor:
        # Launch parallel tasks
        futures = {executor.submit(find_non_zero_in_segment, matrix[start_row:end_row, :]): (start_row, end_row)
                   for start_row, end_row in segments}

        # Gather results
        for future in as_completed(futures):
            segment_info = futures[future]  
            try:
                non_zero_indices = future.result()
                # Adjust indices based on the segment's position in the original matrix
                adjusted_indices = [(r + segment_info[0], c) for r, c in non_zero_indices]
                results.extend(adjusted_indices)
                logging.info(f"Segment {segment_info} processed successfully.")
            except Exception as exc:
                logging.error(f"Segment {segment_info} generated an error: {exc}")

    return results

if __name__ == '__main__':
    non_zero_coordinates = main()
    print(f"Found {len(non_zero_coordinates)} non-zero elements.")
    print("Sample non-zero coordinates:", non_zero_coordinates[:10])





2024-03-28 23:45:06,457 - INFO - Segment (0, 10000) processed successfully.
2024-03-28 23:45:12,396 - INFO - Segment (10000, 20000) processed successfully.
2024-03-28 23:45:18,231 - INFO - Segment (20000, 30000) processed successfully.
2024-03-28 23:45:24,057 - INFO - Segment (30000, 40000) processed successfully.
2024-03-28 23:45:29,880 - INFO - Segment (40000, 50000) processed successfully.
2024-03-28 23:45:35,686 - INFO - Segment (50000, 60000) processed successfully.
2024-03-28 23:45:41,511 - INFO - Segment (60000, 70000) processed successfully.
2024-03-28 23:45:47,315 - INFO - Segment (70000, 80000) processed successfully.
2024-03-28 23:45:53,125 - INFO - Segment (80000, 90000) processed successfully.
2024-03-28 23:45:58,893 - INFO - Segment (90000, 100000) processed successfully.


Found 1000 non-zero elements.
Sample non-zero coordinates: [(21, 7808), (78, 8882), (289, 8340), (404, 18154), (475, 67184), (485, 62548), (614, 15331), (719, 50254), (806, 73291), (1074, 35337)]


## Question 2

#### Realistic Microscope Image Generator

To simulate how a parasite spreads, I initially tried allowing it to move in all directions from a starting point. But this method took too much computer power. So, I changed the approach to make the parasite take bigger steps in any direction, picking which way to go randomly. However, this was still too demanding on the computer. To simplify, I decided the parasite pixels should only move in two directions (left or right) and switch between these after moving a good distance. After being done with a row, I return RLE encoding for that row(start,end). This would still generate a realistic parasite image and which is contonous. To figure out how far it should go in a direction, I used the total area the parasite needs to cover and divided it by the number of rows from where the parasite starts to where it ends. This meant setting a minimum length for each part of its path, starting and ending these parts randomly (start at top half and end at bottom half) but within a framework that allowed for direction changes after significant progress. This method made the simulation much easier for the computer to handle, effectively showing how the parasite would spread.

In [20]:
import random
import math

def decide_walk_direction():
    """Decides the direction of growth for the parasite."""
    #return random.choice(['left', 'right', 'bottom', 'top'])
    return random.choice(['left', 'right'])

def calculate_segment_size(target_area, rows):
    """Calculates the minimum segment size based on the target area and the number of rows."""
    return math.ceil(target_area / rows)

def generate_parasite_segment(start, end, direction, size, min_segment_size):
    """Generates a segment of the parasite based on the given direction, ensuring it meets the minimum segment size."""
    segment_start, segment_end = 0, 0
    segment_length = 0

    while segment_length < min_segment_size:
        if direction == 'right':
            segment_start = random.randint(0, size - min_segment_size)
            segment_end = min(segment_start + min_segment_size + random.randint(0, size - segment_start - min_segment_size), size)
        else:  # direction == 'left'
            segment_end = random.randint(min_segment_size, size) - 1
            segment_start = max(0, segment_end - min_segment_size - random.randint(0, segment_end))

        if segment_end == size:
            segment_end = segment_end - 1
            break
        segment_length = segment_end - segment_start + 1
        #direction top
        #direction bottom
    #RLE start end point return for a row
    return segment_start, segment_end

def Microscope_Image_Generator(size=100000):
    target_area = size * size * 0.25
    start_row = random.randint(0, size//2) #choose a random starting point from first half
    end_row = random.randint(start_row + size//2, size-1) #choose a random ending point from second half
    total_covered_area = 0
    image = [[] for _ in range(size)]

    for row in range(start_row, end_row):
        direction = decide_walk_direction()
        min_segment_size = calculate_segment_size(target_area, end_row - start_row)
        segment_start, segment_end = generate_parasite_segment(row, end_row, direction, size, min_segment_size)
        total_covered_area += segment_end - segment_start + 1
        #Applying RLE along the rows
        image[row].extend([segment_start, segment_end])

    covered_ratio = total_covered_area / (size * size)
    return image, covered_ratio




#### DYE IMAGE GENERATOR
To create a realistic dye image, I couldn't just randomly place dye points; it had to make sense with the underlying structure of the parasite as observed under the microscope. So, starting with the microscope image in RLE format, I chose a few rows at random. In each selected row, I then randomly picked points to apply dye (Enforced a 80% bias to speed up computation but sampling made sure the dye points are not continous), continuing this process until the dye coverage matched the expected concentration within the parasite area.


In [9]:
import random

def generate_dye_image(microscope_image, image_size, has_cancer):
    dye_image = {} # to store coordinates

    # Define the base concentration range. Adjust these values based on your needs.
    base_concentration_range = (0.01, 0.10) if not has_cancer else (0.11, 0.20)
    
    # Extract non-empty segments from the microscope image
    non_empty_segments = [(row_idx, segment[0], segment[1]) for row_idx, segment in enumerate(microscope_image) if segment]
    
    # Calculate the total area of the parasite
    total_parasite_area = sum(segment_end - segment_start + 1 for _, segment_start, segment_end in non_empty_segments)
    
    # Calculate the desired dye coverage based on the total area and selected concentration
    desired_dye_coverage = int(total_parasite_area * random.uniform(*base_concentration_range))
    
    applied_dye_count = 0
    while applied_dye_count < desired_dye_coverage:
        row, start, end = random.choice(non_empty_segments)
        segment_length = end - start + 1
        # Make sure to consider 80 percent of points atleast from original parasite, just to speed up simulation, else it runs too slow
        points_to_apply = random.randint(int(segment_length * 0.8), segment_length)
        selected_points = random.sample(range(start, end + 1), min(points_to_apply, desired_dye_coverage - applied_dye_count))
        
        dye_image.setdefault(row, []).extend(selected_points)
        applied_dye_count += len(selected_points)

    # Simulate leakage outside the parasite body
    # 0.001 * 100,000
    leakage_attempts = random.randint(0, 100)
    while leakage_attempts > 0:
        row = random.randint(0, image_size - 1)
        col = random.randint(0, image_size - 1)
        if len(microscope_image[row]) > 0:
            start, end = microscope_image[row]
            if not start <= col <= end:
                dye_image.setdefault(row, []).append(col)
                applied_dye_count += 1
        leakage_attempts -= 1

    return dye_image





## Question 3

To detect cancer, I iterated each row (key in dye representation) where dye was present and checked if the dye points fell within the parasite's area, as indicated in the microscope image. For every matching point, I incremented a count. After going through all the dye-covered rows, I calculated the percentage of the parasite area covered by dye. If the coverage exceeded 10%, I identified it as a case of cancer. This method effectively assesses the spread of dye within the parasite, aiding in the accurate detection of cancerous cells.

In [63]:
def detect_cancer(microscopic_image, dye_image):
    non_empty_segments = [(row, seg[0], seg[1]) for row, seg in enumerate(microscopic_image) if seg]

    parasite_area = sum(end - start + 1 for _, start, end in non_empty_segments)
    
    dye_covered_area = 0
    for row, cols in dye_image.items():
        if len(microscopic_image[row])>0:
            
            start, end = microscopic_image[row]
            for col in cols:
                if start <= col <= end :
                    dye_covered_area += 1

    coverage_ratio = dye_covered_area / parasite_area if parasite_area > 0 else 0

    return coverage_ratio > 0.1


## Question 4

For an enhanced cancer detection method, I considered create a binary vector, where '1' indicates the presence of dye or the microscope's parasite area, and '0' signifies their absence for each row. By performing logiclal_AND for each corresponding row vector of the parasite area and the dye points, I was able to identify the intersection points where dye is present within the parasite. Summing up these intersection points across all rows gave me the total area covered by dye within the parasite. By comparing this total dyed area against the overall parasite area and checking if the coverage exceeds 10%, I could efficiently determine the presence of cancer. 

In [12]:
import numpy as np

def create_binary_row_for_microscope(row_segments, size=100000):
    row_image = np.zeros(size, dtype=int)
    start, end = row_segments
    #print(start,end)
    row_image[start:end + 1] = 1
    return row_image


def create_binary_row_for_dye(cols, size=100000):
    row_image = np.zeros(size, dtype=int)
    row_image[cols] = 1
    return row_image

def improved_detect_cancer(microscopic_image, dye_image, size=100000):
    total_parasite_area = 0
    dye_covered_area = 0

    for row_idx, segments in enumerate(microscopic_image):
        # Create binary representation for the microscope row
        #print(row_idx)
        if len(segments) > 0 :
            micro_row_image = create_binary_row_for_microscope(segments, size)
            total_parasite_area += micro_row_image.sum()
    
            # Create binary representation for the dye row if it exists in the dye_image
            if row_idx in dye_image:
                dye_row_image = create_binary_row_for_dye(dye_image[row_idx], size)
                # Compute intersection
                intersection = np.logical_and(micro_row_image, dye_row_image)
                dye_covered_area += intersection.sum()

    coverage_ratio = dye_covered_area / total_parasite_area if total_parasite_area > 0 else 0


    return coverage_ratio > 0.1


## Question 5

#### Alternative Techniques for Microscope

Boundary Representation (B-Rep): </br>
For images where the parasite forms a continuous shape, storing just the boundaries of these shapes can be more efficient than storing information about every pixel inside. 
Although it will be highly compressed in compariosn to RLE as we will just save boundary, However computing the exact boundary will be complex, maybe algorithms like Canny edge detector might come in handy.

#### Alternative Techniques for Dye Image

Quadtree for Dye Image :
A quadtree is a tree data structure in which each node has exactly four children. Quadtrees are particularly suited for partitioning a two-dimensional space by recursively subdividing it into four quadrants or regions. 
Quadtrees can adaptively represent the image by subdividing regions with high dye concentration (i.e., the parasite body) into smaller cells, while using larger cells for regions with low or no dye (i.e., the background). This allows for a compact representation that focuses storage on the areas of interest.
It is very efficient in both storage and querying. However while construction for such a large image was getting computationally expensive on my pc so I had to resort to COO, but it is much better representation for Dye image.


In [25]:
class QuadTreeNode:
    def __init__(self, x, y, size, color):
        self.x = x
        self.y = y
        self.size = size
        self.color = color
        self.children = [None, None, None, None]

    def is_leaf(self):
        return all(child is None for child in self.children)

    def split(self):
        half_size = self.size // 2
        self.children = [
            QuadTreeNode(self.x, self.y, half_size, self.color),
            QuadTreeNode(self.x + half_size, self.y, half_size, self.color),
            QuadTreeNode(self.x, self.y + half_size, half_size, self.color),
            QuadTreeNode(self.x + half_size, self.y + half_size, half_size, self.color)
        ]

    def __str__(self):
        return f"Node(x={self.x}, y={self.y}, size={self.size}, color={self.color})"

def build_quadtree(image, min_size=1):
    def _build_quadtree(x, y, size):
        if size < min_size:
            return QuadTreeNode(x, y, size, image[y][x])

        half_size = size // 2
        node = QuadTreeNode(x, y, size, image[y][x])

        # Check if the node should be split
        if any(image[y][x] != image[y2][x2] for y2 in range(y, y + size) for x2 in range(x, x + size)):
            node.split()
            for i in range(4):
                child_x = x + (i % 2) * half_size
                child_y = y + (i // 2) * half_size
                node.children[i] = _build_quadtree(child_x, child_y, half_size)

        return node

    return _build_quadtree(0, 0, len(image))



#root = build_quadtree(image)

def find_regions_with_ones(node):
    regions = []

    if node.is_leaf():
        if node.color == 1:
            regions.append((node.x, node.y, node.size, node.size))
        return regions

    for child in node.children:
        if child is not None:
            regions.extend(find_regions_with_ones(child))

    return regions


####### Dummy Image
image = [
    [0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0]
]

root = build_quadtree(image)
regions_with_ones = find_regions_with_ones(root)
    

In [ ]:
#### class QuadTreeNode:
    def __init__(self, x, y, size, color):
        self.x = x
        self.y = y
        self.size = size
        self.color = color
        self.children = [None, None, None, None]

    def is_leaf(self):
        return all(child is None for child in self.children)

    def split(self):
        half_size = self.size // 2
        self.children = [
            QuadTreeNode(self.x, self.y, half_size, self.color),
            QuadTreeNode(self.x + half_size, self.y, half_size, self.color),
            QuadTreeNode(self.x, self.y + half_size, half_size, self.color),
            QuadTreeNode(self.x + half_size, self.y + half_size, half_size, self.color)
        ]

    def __str__(self):
        return f"Node(x={self.x}, y={self.y}, size={self.size}, color={self.color})"

def build_quadtree(image, min_size=1):
    def _build_quadtree(x, y, size):
        if size < min_size:
            return QuadTreeNode(x, y, size, image[y][x])

        half_size = size // 2
        node = QuadTreeNode(x, y, size, image[y][x])

        # Check if the node should be split
        if any(image[y][x] != image[y2][x2] for y2 in range(y, y + size) for x2 in range(x, x + size)):
            node.split()
            for i in range(4):
                child_x = x + (i % 2) * half_size
                child_y = y + (i // 2) * half_size
                node.children[i] = _build_quadtree(child_x, child_y, half_size)

        return node

    return _build_quadtree(0, 0, len(image))



#root = build_quadtree(image)

def find_regions_with_ones(node):
    regions = []

    if node.is_leaf():
        if node.color == 1:
            regions.append((node.x, node.y, node.size, node.size))
        return regions

    for child in node.children:
        if child is not None:
            regions.extend(find_regions_with_ones(child))

    return regions


####### Dummy Image
image = [
    [0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 0]
]

root = build_quadtree(image)
regions_with_ones = find_regions_with_ones(root)

for x, y, width, height in regions_with_ones:
    print(f"Region: x={x}, y={y}, width={width}, height={height}")
    

#### Actual Runtime and Computation Cost for Realistic Images

In [15]:
import sys

image_size = 100000
cancer_status = True

microscope_image, covered_ratio = Microscope_Image_Generator(image_size)
dye_image = generate_dye_image(microscope_image, image_size, cancer_status)

print(f"Size of microscope image: {sys.getsizeof(microscope_image)} bytes")
print(f"Size of dye image: {sys.getsizeof(dye_image)} bytes")

Size of microscope image: 824456 bytes
Size of dye image: 295000 bytes


Size of microscope image orginally as 100000 x100000, is now compressed to 0.824456 MB </br>
Size of Dye image orginally as 100000 x100000, is now compressed to 0.295 MB

In [14]:
import time
start_time = time.time()

val = improved_detect_cancer(microscope_image,dye_image)
end_time = time.time()
execution_time = end_time - start_time
print("Parasite Detected with cancer:", val)
print("Execution Time:", execution_time, "seconds")

Parasite Detected with cancer: True
Execution Time: 39.4471321105957 seconds


As we passed status to dye_image as True, our cancer detection method also returns true.

## Question 6

Stackover flow, medium blogs for Quadtree, RLE and random walks details. </br>
Chatgpt, Perplexity, Copilot for debugging and code improvements.</br>
LLM as a tool was not used to solve this challenge directly, only it was used to provide suggestions or research.</br>